In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import torch

# 创建数据库引擎
database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

# define hyper paras
use_standard_scaler = True
save_imputed_data = False
notebook_mode = 'train'
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

model_name = 'PatchTFT'
saved_table = 'sh_customs_daily_pred'
version = '0.5.0'

In [2]:
query = "select date as ds,global_entry,hkmo_entry,tw_entry from sh_customs_daily_imputed"
data = pd.read_sql_query(query, engine)
original_data = data.copy()

In [5]:
from sklearn.preprocessing import MinMaxScaler

scalers = {}

for column in data.columns:
    if column not in ['ds']:
        scaler = MinMaxScaler()
        data[column] = scaler.fit_transform(data[column].values.reshape(-1, 1)).reshape(
            -1)
        scalers[column] = scaler

In [6]:
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, FEDformer, PatchTST

horizon = 365  # 1 year
models = [Informer(h=horizon,  # Forecasting horizon
                   input_size=horizon,  # Input size
                   max_steps=1000,  # Number of training iterations
                   val_check_steps=100,  # Compute validation loss every 100 steps
                   early_stop_patience_steps=3),  # Stop training if validation loss does not improve
          Autoformer(h=horizon,
                     input_size=horizon,
                     max_steps=1000,
                     val_check_steps=100,
                     early_stop_patience_steps=3),
          PatchTST(h=horizon,
                   input_size=horizon,
                   max_steps=1000,
                   val_check_steps=100,
                   early_stop_patience_steps=3),
          ]

Seed set to 1
Seed set to 1
Seed set to 1


In [7]:
# melting,转换为长格式
df_long = pd.melt(data, id_vars=['ds'], var_name='unique_id', value_name='y')
df_long['ds'] = pd.to_datetime(df_long.ds)

## 使用cross validation选择模型

使用RTX 4070Ti GPU 训练3小时

In [8]:
nf = NeuralForecast(
    models=models,
    freq='D')

n_time = len(df_long)
val_size = int(.1 * n_time)
test_size = int(.1 * n_time)

Y_hat_df = nf.cross_validation(df=df_long, val_size=val_size, test_size=test_size, n_windows=None)

Epoch 99: 100%|██████████| 1/1 [00:05<00:00,  0.18it/s, v_num=3, train_loss_step=0.121, train_loss_epoch=0.122]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 199: 100%|██████████| 1/1 [00:05<00:00,  0.18it/s, v_num=3, train_loss_step=0.107, train_loss_epoch=0.107, valid_loss=0.137]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 299: 100%|██████████| 1/1 [00:05<00:00,  0.18it/s, v_num=3, train_loss_step=0.0983, train_loss_epoch=0.0982, valid_loss=0.140]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 399: 100%|██████████| 1/1 [00:05<00:00,  0.18it/s, v_num=3, train_loss_step=0.0925, train_loss_epoch=0.0924, valid_loss=0.139]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 499: 100%|██████████| 1/1 [00:06<00:00,  0.17it/s, v_num=3, train_loss_step=0.0885, train_loss_epoch=0.0888, valid_loss=0.131]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 599: 100%|██████████| 1/1 [00:05<00:00,  0.18it/s, v_num=3, train_loss_step=0.0861, train_loss_epoch=0.0868, valid_loss=0.

OutOfMemoryError: CUDA out of memory. Tried to allocate 274.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.80 GiB is allocated by PyTorch, and 387.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
nf.save('../models/Informer_1.0.0',model_index=[0],save_dataset=True)

In [14]:
informer = NeuralForecast.load('../models/Informer_1.0.0')
informer.predict()

Seed set to 1


In [18]:
informer.

Exception: You must fit the model before predicting.